In [1]:
# import tensorflow as tf
import tensorflow as tf
tf.compat.v1.disable_v2_behavior() # Use only for Deep SHAP
tf.compat.v1.disable_eager_execution() # Use only for Deep SHAP
import tensorflow.keras as keras
import numpy as np
import shap
import sys, os
import time
from tqdm.notebook import tqdm

from tensorflow.keras.layers import (Input, Layer, Dense)
from tensorflow.keras.models import Model, Sequential
import tensorflow.keras.backend as K

from tf_explain.utils.image import transform_to_normalized_grayscale

import argparse
import pickle



# IMPORTANT: SET RANDOM SEEDS FOR REPRODUCIBILITY
os.environ['PYTHONHASHSEED'] = str(420)
import random
random.seed(420)
np.random.seed(420)
tf.random.set_seed(420)

#Select GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

Instructions for updating:
non-resource variables are not supported in the long term


RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

## Load Images, Predictions, and Labels

In [2]:
images_dir = os.path.join(os.getcwd(), 'images')
img = np.load(os.path.join(images_dir, 'processed_images.npy'), allow_pickle=True)
labels = np.load(os.path.join(images_dir, 'labels.npy'), allow_pickle=True)
preds = np.load(os.path.join(images_dir, 'predictions.npy'), allow_pickle=True)

## Load Model

In [3]:
from tensorflow.keras.applications.resnet50 import ResNet50

INPUT_SHAPE = (224, 224, 3)
base_model = ResNet50(
    include_top=False, weights='imagenet', 
    input_shape=INPUT_SHAPE, pooling='avg'
)
base_model.trainable = True

model_input = Input(shape=INPUT_SHAPE, name='input')

net = base_model(model_input)
out = Dense(10, activation='softmax')(net)

model = Model(model_input, out)

model_weights_path = 'model/20210514_14_39_21/model_weights.h5'

model.load_weights(model_weights_path)
model.trainable = False

# Gradient Based Explainations

## Grad CAM

### Model Surgery

In [4]:
model_flat = Sequential()
conv_model = Model(
    model.input, model.layers[1].get_layer('conv5_block3_out').get_output_at(1)
)
model_flat.add(conv_model)
model_flat.add(model.layers[1].get_layer('avg_pool'))
model_flat.add(model.layers[-1])

### Explain

In [7]:
from tf_explain.core.grad_cam import GradCAM
import cv2

gcam = GradCAM()
layer_name = gcam.infer_grad_cam_target_layer(model_flat)
cams = []
t = time.time()
grad_model = tf.keras.models.Model(
    model_flat.inputs, [model_flat.get_layer(layer_name).get_output_at(0), model_flat.output]
)

cams = []
for i in tqdm(range(20)): #mini-batch
    with tf.GradientTape() as tape:
        inputs = tf.cast(img[i*50:(i+1)*50], tf.float32)
        tape.watch(inputs)
        conv_outputs, predictions = grad_model(inputs)
        loss = K.max(predictions, 1)

    grads = tape.gradient(loss, conv_outputs)
    grads = (
        tf.cast(conv_outputs > 0, "float32")
        * tf.cast(grads > 0, "float32")
        * grads
    )
    
    cam = tf.stack(GradCAM.generate_ponderated_output(conv_outputs, grads))
    cams.append(cam)
    
cams = np.array([cv2.resize(cam, (224, 224)) for cam in tf.concat(cams, 0).numpy()])

explaining_time = time.time() - t

  0%|          | 0/20 [00:00<?, ?it/s]

In [10]:
save_dir = 'gradcam'
model_dir = os.path.join(os.getcwd(), save_dir)
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)
    
cams.dump(os.path.join(model_dir, 'explanations.npy'))

with open(os.path.join(model_dir, 'explaining_time.pkl'), 'wb') as f:
    pickle.dump(explaining_time, f)

## SmoothGrad

In [26]:
from tf_explain.core.smoothgrad import SmoothGrad
import itertools

t = time.time()

smoothgrads = []
for i in tqdm(range(100)):
    sg = SmoothGrad()
    num_samples = 5
    noisy_images = sg.generate_noisy_images(img[i*10:(i+1)*10], num_samples=num_samples, noise=1.0)

    expected_output = tf.one_hot(
        list(itertools.chain(*[[yp.argmax()] * num_samples for yp in preds[i*10:(i+1)*10]])),
        10,
        on_value=None,
        off_value=None,
    )

    with tf.GradientTape() as tape:
        inputs = tf.cast(noisy_images, tf.float32)
        tape.watch(inputs)
        predictions = model(inputs)
        loss = tf.keras.losses.categorical_crossentropy(
            expected_output, predictions
        )

    grads = tape.gradient(loss, inputs)

    grads_per_image = tf.reshape(grads, (-1, num_samples, *grads.shape[1:]))
    smoothed_gradients = tf.reduce_mean(grads_per_image, axis=1)

    grayscale_gradients = transform_to_normalized_grayscale(
            tf.abs(smoothed_gradients)
        ).numpy()

    smoothgrads.append(grayscale_gradients)
    
explaining_time = time.time() - t

  0%|          | 0/100 [00:00<?, ?it/s]

In [32]:
save_dir = 'smoothgrad'
model_dir = os.path.join(os.getcwd(), save_dir)
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)
    
smoothgrads = np.vstack(smoothgrads)
smoothgrads.dump(os.path.join(model_dir, 'explanations.npy'))

with open(os.path.join(model_dir, 'explaining_time.pkl'), 'wb') as f:
    pickle.dump(explaining_time, f)

## Integrated Gradients

In [18]:
from tf_explain.core.integrated_gradients import IntegratedGradients

t = time.time()
igrads = []
n_steps = 10
for i in tqdm(range(200)): #mini-batch
    ig = IntegratedGradients()
    interpolated_images = ig.generate_interpolations(
            img[i*5:(i+1)*5], n_steps=n_steps
        )

    with tf.GradientTape() as tape:
        inputs = tf.cast(interpolated_images, tf.float32)
        tape.watch(inputs)
        predictions = model(inputs)
        loss = K.max(predictions, 1)

    grads = tape.gradient(loss, inputs)
    grads_per_image = tf.reshape(grads, (-1, n_steps, *grads.shape[1:]))

    integrated_gradients = tf.reduce_mean(grads_per_image, axis=1)

    grayscale_integrated_gradients = transform_to_normalized_grayscale(
            tf.abs(integrated_gradients)
        ).numpy()

    igrads.append(grayscale_integrated_gradients)

    
explaining_time = time.time() - t

  0%|          | 0/200 [00:00<?, ?it/s]

In [20]:
save_dir = 'integratedgradients'
model_dir = os.path.join(os.getcwd(), save_dir)
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)
    
igrads = np.vstack(igrads)
igrads.dump(os.path.join(model_dir, 'explanations.npy'))

with open(os.path.join(model_dir, 'explaining_time.pkl'), 'wb') as f:
    pickle.dump(explaining_time, f)

## SHAP Deep Explainer

In [4]:
import shap 

deepshap = shap.DeepExplainer(model=model, 
                              data=np.expand_dims(np.zeros_like(img[0]), 0))

keras is no longer supported, please use tf.keras instead.


In [5]:
t = time.time()
shap_values = deepshap.shap_values(img)
explaining_time = time.time() - t

In [6]:
save_dir = 'deepshap'
model_dir = os.path.join(os.getcwd(), save_dir)
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

with open(os.path.join(model_dir, 'explaining_time.pkl'), 'wb') as f:
    pickle.dump(explaining_time, f)
    
with open(os.path.join(model_dir, 'shap_values.pkl'), 'wb') as f:
    pickle.dump(shap_values, f)